In [1]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 24.1 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=f9cc1f0529140cc6019ee9d530d7b20dd79ff3d77842a85447c224181e60c9b9
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [3]:
!pip install beautifulsoup4

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [6]:
%cd drive
!ls

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping
datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [7]:
%cd MyDrive

[Errno 2] No such file or directory: 'MyDrive'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [8]:
!ls

datasets_updated
scrape_wikipedia_arabic_1982_Lebanon_War.ipynb
scrape_wikipedia_arabic_Alexander_Haig.ipynb
scrape_wikipedia_arabic_Assassination_of_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_August_1982_Lebanese_presidential_election.ipynb
scrape_wikipedia_arabic_Bachir_Gemayel.ipynb
scrape_wikipedia_arabic_Israeli_Lebanese_conflict.ipynb
scrape_wikipedia_arabic_Israeli_occupation_of_Southern_Lebanon.ipynb
scrape_wikipedia_arabic_Operation_Mole_Cricket_19.ipynb
scrape_wikipedia_arabic_Sabra_and_Shatila_massacre.ipynb
scrape_wikipedia_arabic_September_1982_Lebanese_presidential_election.ipynb


In [9]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [10]:
page = wptools.page('Israeli_occupation_of_Southern_Lebanon')
page.get_parse()

en.wikipedia.org (parse) Israeli_occupation_of_Southern_Lebanon
Israeli occupation of Southern Lebanon (en) data
{
  infobox: <dict(24)> conventional_long_name, common_name, nation,...
  iwlinks: <list(1)> https://he.wikipedia.org/wiki/%D7%A8%D7%A6%D7...
  pageid: 36734475
  parsetree: <str(24431)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Israeli occupation of Southern Lebanon
  wikibase: Q2910225
  wikidata_url: https://www.wikidata.org/wiki/Q2910225
  wikitext: <str(18984)> {{Short description|1982–2000 military oc...
}


In [11]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Israeli_occupation_of_Southern_Lebanon",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Israeli occupation of Southern Lebanon')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1978 South Lebanon conflict
1978 Southern Lebanon conflict
1982 Lebanon War
1999 Israeli legislative election
Abbas Musawi
Abdel Karim Obeid
Annexation of the Jordan Valley
Antoine Lahad
Arabic language
Area C (West Bank)
Army of Free Lebanon
Awali (river)
Beirut
Blockade of the Gaza Strip
Blue Line (Lebanon)
Borders of Israel
Central Intelligence Agency
Christianity in Lebanon
Christopher Mayhew
Coastal Road massacre
David Hirst (journalist)
Declarations of State Land in the West Bank
Dennis Walters
Doi (identifier)
East Jerusalem
Ehud Barak
Fatima Gate
Free Lebanon State
Galilee Medical Center
Golan Heights
Good Fence
Haifa
Hasbaya
Hebrew language
Hezbollah
Hizbullah
Human rights violations against Palestinians by Israel
ISBN (identifier)
International law and Israeli settlements
Islamic extremism
Israel
Israel Defence Force
Israel Defense Forces
Israel and apartheid
Israeli-occupied territories
Israeli Air Force
Israeli Navy
Israeli criticism of the occupation of Palestine
Is

In [12]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [13]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [14]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [15]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1978 Southern Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1999 Israeli legislative election
en.wikipedia.org (parse) Abbas Musawi
en.wikipedia.org (imageinfo) File:Abbas al Musawi (cropped).jpg
en.wikipedia.org (parse) Abdel Karim Obeid
en.wikipedia.org (imageinfo) File:השייח' עובייד במטוס לגרמניה (cr...
en.wikipedia.org (parse) Annexation of the Jordan Valley
en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Area C (West Bank)
en.wikipedia.org (parse) Army of Free Lebanon
en.wikipedia.org (imageinfo) File:Former Flag of the Lebanese Army.svg


['جيش لبنان الحر']
-----------------
name:  ['جيش لبنان الحر']
founders:  
ideology:  ['Anti-communism', 'Christian nationalism', 'Phoenicianism']
position:  -
START
Cannot translate phrase `جيش لبنان الحر` to arabic
END
START
ideology:  Anti-communism
ideology ar:  معاداة الشيوعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B4%D9%8A%D9%88%D8%B9%D9%8A%D8%A9
ideology:  Christian nationalism
ideology ar:  قومية مسيحية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%85%D8%B3%D9%8A%D8%AD%D9%8A%D8%A9
ideology:  Phoenicianism
ideology ar:  قومية فينيقية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%81%D9%8A%D9%86%D9%8A%D9%82%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Awali (river)
en.wikipedia.org (imageinfo) File:Awali River Route.png
en.wikipedia.org (parse) Beirut
en.wikipedia.org (parse) Blockade of the Gaza Strip
en.wikipedia.org (parse) Blue Line (Lebanon)
en.wikipedia.org (parse) Borders of Israel
en.wikipedia.org (parse) Central Intelligence Agency
en.wikipedia.org (parse) Christianity in Lebanon
en.wikipedia.org (imageinfo) File:HarissaVirginOfLebanon.jpg
en.wikipedia.org (parse) Christopher Mayhew
en.wikipedia.org (parse) Coastal Road massacre
en.wikipedia.org (imageinfo) File:הפיגוע בכביש החוף 1978.jpg
en.wikipedia.org (parse) David Hirst (journalist)
en.wikipedia.org (parse) Declarations of State Land in the West Bank
en.wikipedia.org (parse) Dennis Walters
en.wikipedia.org (parse) Doi (identifier)
en.wikipedia.org (imageinfo) File:DOI logo.svg
en.wikipedia.org (parse) East Jerusalem
en.wikipedia.org (parse) Ehud Barak
en.wikipedia.org (imageinfo) File:Ehud Barak 2016 - Herzliya Conf...
en.wikipedia.org (parse) 

''Vacant''
[[Naim Qassem]]
[[Hashem Safieddine]]
['Islamic Resistance in Lebanon']
-----------------
name:  ['Islamic Resistance in Lebanon']
founders:  Vacant,Naim Qassem,Hashem Safieddine,
ideology:  ['Anti-LGBT', 'Antisemitism', 'Anti-Zionism', 'Anti-imperialism', 'Shia Islamism', 'Anti-West', 'Unbulleted list', 'Shia jihadism', 'Populism', 'Khomeinism', 'Pan-Islamism', 'Islamic nationalism', 'Sectarianism']
position:  ['Syncretic']
START
name:  Islamic Resistance in Lebanon
name ar:  حزب الله 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%84%D9%87
END
START
Cannot find the link: http://en.wikipedia.org/wiki/AntiLGBT
ideology:  Anti-LGBT
ideology ar:  ضد المثليين (مترجمه)
ideo ar link:  -1
ideology:  Antisemitism
ideology ar:  معاداة السامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B3%D8%A7%D9%85%D9%8A%D8%A9
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.wik

en.wikipedia.org (parse) Hizbullah


...

General Secretary
['Popular Front for the Liberation of Palestine – General Command']
-----------------
name:  ['Popular Front for the Liberation of Palestine – General Command']
founders:  General Secretary,
ideology:  ['Arab nationalism', 'Anti-Zionism', 'Palestinian nationalism']
position:  ['Left-wing to right-wing']
START
name:  Popular Front for the Liberation of Palestine – General Command
name ar:  الجبهة الشعبية لتحرير فلسطين – القيادة العامة 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AC%D8%A8%D9%87%D8%A9_%D8%A7%D9%84%D8%B4%D8%B9%D8%A8%D9%8A%D8%A9_%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86_%E2%80%93_%D8%A7%D9%84%D9%82%D9%8A%D8%A7%D8%AF%D8%A9_%D8%A7%D9%84%D8%B9%D8%A7%D9%85%D8%A9
END
START
ideology:  Arab nationalism
ideology ar:  قومية عربية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D8%B9%D8%B1%D8%A8%D9%8A%D8%A9
ideology:  Anti-Zionism
ideology ar:  معاداة الصهيونية 
ideo ar link:  https://ar.

en.wikipedia.org (parse) Popular Guard
en.wikipedia.org (imageinfo) File:Flag of the Lebanese Communist ...
en.wikipedia.org (parse) Prime Minister of Israeli occupation of ...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Prime Minister of the Israeli occupation...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Prime minister 

[[Saad Haddad]] (1977–1984)
['South Lebanon Army']
-----------------
name:  ['South Lebanon Army']
founders:  ,
ideology:  ['Maronite politics', 'Secularism', 'Multiconfessionalism', 'Anti-Palestinianism', 'Anti-communism', 'Factions:', 'Muslim-Christian Unity', 'Lebanese nationalism', 'Zionism']
position:  -
START
name:  South Lebanon Army
name ar:  جيش لبنان الجنوبي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AC%D9%8A%D8%B4_%D9%84%D8%A8%D9%86%D8%A7%D9%86_%D8%A7%D9%84%D8%AC%D9%86%D9%88%D8%A8%D9%8A
END
START
ideology:  Maronite politics
ideology ar:  مارونية سياسية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%A7%D8%B1%D9%88%D9%86%D9%8A%D8%A9_%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A%D8%A9
ideology:  Secularism
ideology ar:  علمانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85%D8%A7%D9%86%D9%8A%D8%A9
ideology:  Multiconfessionalism
ideology ar:  التعددية المذهبية (مترجمه)
ideo ar link:  -1
Cannot find the link: http://en.wikipedia.org/wiki/AntiPalestinianism
id

en.wikipedia.org (parse) South Lebanon conflict (1985–2000)
en.wikipedia.org (imageinfo) File:South Lebanon conflict montage.png
en.wikipedia.org (parse) South Lebanon security belt administration
en.wikipedia.org (parse) Southern Lebanon
en.wikipedia.org (parse) Status of Jerusalem
en.wikipedia.org (parse) Status of territories occupied by Israel...
en.wikipedia.org (parse) Status of the Golan Heights
en.wikipedia.org (parse) Sunni Islam
en.wikipedia.org (parse) Syria
en.wikipedia.org (parse) Tel Aviv
en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The New York Times, January 13,...
en.wikipedia.org (parse) The Times of Israel
en.wikipedia.org (imageinfo) File:Times of Israel office, Jerusalem.JPG
en.wikipedia.org (parse) UNIFIL
en.wikipedia.org (imageinfo) File:United Nations Interim Force in...
en.wikipedia.org (parse) United Nations
en.wikipedia.org (parse) United Nations Interim Force in Lebanon
en.wikipedia.org (imageinfo) File:United Nations Interim

skipping Wikipedia:Copying within Wikipedia
skipping Wikipedia:Translation
skipping Template:Cite web
skipping Template:Israeli-occupied territories
skipping Template:Israeli occupations navbox
skipping Template talk:Israeli-occupied territories
skipping Template talk:Israeli occupations navbox
skipping Help:Edit summary
skipping Help:Interlanguage links
skipping Category:CS1 maint: multiple names: authors list
skipping Category:History articles needing translation from Hebrew Wikipedia
skipping Category:Mathematics and art
skipping Category:States and territories disestablished in 2000
skipping Category:States and territories disestablished in the 2000
skipping Category:States and territories established in 1982
skipping Category:States and territories established in the 1982
skipping Category:Use Oxford spelling from April 2022
skipping Category:Use dmy dates from April 2022


In [16]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,جيش لبنان الحر,"Anti-communism,Christian nationalism,Phoenicia...",-,
1,Islamic Resistance in Lebanon,"Anti-LGBT,Antisemitism,Anti-Zionism,Anti-imper...",Syncretic,"Vacant,Naim Qassem,Hashem Safieddine,"
2,Islamic Resistance in Lebanon,"Anti-LGBT,Antisemitism,Anti-Zionism,Anti-imper...",Syncretic,"Vacant,Naim Qassem,Hashem Safieddine,"
3,Lebanese Communist Party,"Marxism–Leninism,Anti-Zionism,Nonsectarianism,...",Far-left,
4,Lebanese Front,"Phoenicianism,Christian nationalism,Anti-Pales...",-,
5,Palestine Liberation Organization,"Pan-Arabism,August 2024,Anti-Zionism,Secularis...",Left-wing,"Mahmoud Abbas,"
6,Popular Front for the Liberation of Palestine ...,"Arab nationalism,Anti-Zionism,Palestinian nati...",Left-wing to right-wing,"General Secretary,"
7,South Lebanon Army,"Maronite politics,Secularism,Multiconfessional...",-,","


In [17]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,,"معاداة الشيوعية ,قومية مسيحية ,قومية فينيقية",-,
1,حزب الله,"ضد المثليين (مترجمه),معاداة السامية ,معاداة ال...",توفيق بين الأديان,"إشغال ,هاشم صفي الدين"
2,حزب الله,"ضد المثليين (مترجمه),معاداة السامية ,معاداة ال...",توفيق بين الأديان,"إشغال ,هاشم صفي الدين"
3,الحزب الشيوعي اللبناني,"ماركسية لينينية ,معاداة الصهيونية ,اللاطائفية ...",أقصى اليسار (مترجمه),
4,الجبهة اللبنانية,"قومية فينيقية ,قومية مسيحية ,معاداة فلسطين (مت...",-,
5,منظمة التحرير الفلسطينية,"القومية العربية (مترجمه),أغسطس ,معاداة الصهيون...",يسارية,محمود عباس
6,الجبهة الشعبية لتحرير فلسطين – القيادة العامة,"قومية عربية ,معاداة الصهيونية ,وطنية فلسطينية",من اليسار إلى اليمين (مترجمه),أمين عام
7,جيش لبنان الجنوبي,"مارونية سياسية ,علمانية ,التعددية المذهبية (مت...",-,


In [18]:
datasets_dir = 'datasets_updated/Israeli_occupation_of_Southern_Lebanon/'

In [19]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [20]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [21]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1978 Southern Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1999 Israeli legislative election
en.wikipedia.org (parse) Abbas Musawi
en.wikipedia.org (imageinfo) File:Abbas al Musawi (cropped).jpg


['Abbas al-Musawi']
name_inserted before: ['Abbas al-Musawi']
name_inserted after: Abbas al-Musawi
-----------------
name:  ['Abbas al-Musawi']
native_name:  ['عباس الموسوي', 'nobold']
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  عباس الموسوي (عالم دين) 
name ar:  عباس الموسوي (عالم دين) 
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Abdel Karim Obeid
en.wikipedia.org (imageinfo) File:השייח' עובייד במטוס לגרמניה (cr...


['Abdel Karim Obeid']
name_inserted before: ['Abdel Karim Obeid']
name_inserted after: Abdel Karim Obeid
-----------------
name:  ['Abdel Karim Obeid']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
Cannot translate phrase `Abdel Karim Obeid` to arabic
name ar:  -1
name ar:  -1
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Annexation of the Jordan Valley
en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg


...

['William Richard Higgins']
name_inserted before: ['William Richard Higgins']
name_inserted after: William Richard Higgins
-----------------
name:  ['William Richard Higgins']
native_name:  -
nickname:  ['Rich']
religion:  -
party:  -
nationality:  -
Cannot find the link: http://en.wikipedia.org/wiki/William R Higgins
name ar:  وليام ر هيجينز (مترجمه)
name ar:  وليام ر هيجينز (مترجمه)
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Yitzhak Rabin
en.wikipedia.org (imageinfo) File:Flickr - Israel Defense Forces ...


['Yitzhak Rabin']
name_inserted before: ['Yitzhak Rabin']
name_inserted after: Yitzhak Rabin
-----------------
name:  ['Yitzhak Rabin']
native_name:  ['nobold', 'יִצְחָק רַבִּין']
nickname:  -
religion:  -
party:  ['Alignment, Labor Party']
nationality:  ['Israeli']
name ar:  إسحاق رابين 
START
Cannot find the link: http://en.wikipedia.org/wiki/Alignment Labor Party
political party:  Alignment, Labor Party
political party ar:  حزب العمال المحاذاة (مترجمه)
political party ar link:  -1
END
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  إسحاق رابين 
religion ar:  ['حزب العمال المحاذاة (مترجمه)']
party ar:  ['Alignment, Labor Party']
nationality ar:  ['إسرائيلي (مترجمه)']


en.wikipedia.org (parse) Talk:Israeli occupation of Southern Lebanon


skipping Wikipedia:Copying within Wikipedia
skipping Wikipedia:Translation
skipping Template:Cite web
skipping Template:Israeli-occupied territories
skipping Template:Israeli occupations navbox
skipping Template talk:Israeli-occupied territories
skipping Template talk:Israeli occupations navbox
skipping Help:Edit summary
skipping Help:Interlanguage links
skipping Category:CS1 maint: multiple names: authors list
skipping Category:History articles needing translation from Hebrew Wikipedia
skipping Category:Mathematics and art
skipping Category:States and territories disestablished in 2000
skipping Category:States and territories disestablished in the 2000
skipping Category:States and territories established in 1982
skipping Category:States and territories established in the 1982
skipping Category:Use Oxford spelling from April 2022
skipping Category:Use dmy dates from April 2022


In [22]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abbas al-Musawi,-,-,-
1,Abdel Karim Obeid,-,-,-
2,Antoine Lahad,-,-,-
3,The Lord Mayhew,unbulleted list,-,-
4,David Hirst,-,British,-
5,Dennis Walters,Conservative,-,-
6,Ehud Barak,Israel Democratic Party,-,-
7,Jim Muir,-,-,-
8,Saad Haddad,-,-,-
9,Shaul Mofaz,-,-,-


In [23]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,-,عباس الموسوي (عالم دين)
1,-,-,-,-1
2,-,-,-,أنطوان لحد
3,-,-,قائمة غير منقوطة (مترجمه),كريستوفر مايوه (مترجمه)
4,-,بريطاني (مترجمه),-,ديفيد هيرست صحفي (مترجمه)
5,-,-,سياسة محافظة,دينيس والترز (مترجمه)
6,-,-,الحزب الديمقراطي الإسرائيلي (مترجمه),إيهود باراك
7,-,-,-,جيم موير (مترجمه)
8,-,-,-,سعد حداد
9,-,-,-,شاؤول موفاز


In [24]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [25]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [26]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1978 Southern Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1999 Israeli legislative election
en.wikipedia.org (parse) Abbas Musawi
en.wikipedia.org (imageinfo) File:Abbas al Musawi (cropped).jpg
en.wikipedia.org (parse) Abdel Karim Obeid
en.wikipedia.org (imageinfo) File:השייח' עובייד במטוס לגרמניה (cr...
en.wikipedia.org (parse) Annexation of the Jordan Valley
en.wikipedia.org (parse) Antoine Lahad
en.wikipedia.org (imageinfo) File:Antoine Lahad.jpg
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Area C (West Bank)
en.wikipedia.org (parse) Army of Free Lebanon
en.wikipedia.org (imageinfo) File:Former Flag of the Lebanese Army.svg
en.wikipedia.org (parse) Awali (river)
en.wikipe

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Blockade of the Gaza Strip
en.wikipedia.org (parse) Blue Line (Lebanon)
en.wikipedia.org (parse) Borders of Israel
en.wikipedia.org (parse) Central Intelligence Agency
en.wikipedia.org (parse) Christianity in Lebanon
en.wikipedia.org (imageinfo) File:HarissaVirginOfLebanon.jpg
en.wikipedia.org (parse) Christopher Mayhew
en.wikipedia.org (parse) Coastal Road massacre
en.wikipedia.org (imageinfo) File:הפיגוע בכביש החוף 1978.jpg
en.wikipedia.org (parse) David Hirst (journalist)
en.wikipedia.org (parse) Declarations of State Land in the West Bank
en.wikipedia.org (parse) Dennis Walters
en.wikipedia.org (parse) Doi (identifier)
en.wikipedia.org (imageinfo) File:DOI logo.svg
en.wikipedia.org (parse) East Jerusalem
en.wikipedia.org (parse) Ehud Barak
en.wikipedia.org (imageinfo) File:Ehud Barak 2016 - Herzliya Conf...
en.wikipedia.org (parse) Fatima Gate
en.wikipedia.org (parse) Free Lebanon State


-----------------
name:  Southern Lebanon
subdivision_type:  -
subdivision_name:  -
name ar:  جنوب لبنان 


en.wikipedia.org (parse) Galilee Medical Center
en.wikipedia.org (parse) Golan Heights


...

-----------------
name:  Syria
subdivision_type:  -
subdivision_name:  -
name ar:  سوريا 


en.wikipedia.org (parse) Tel Aviv


-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['Israel']
skipping Tel Aviv because name is not found


en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The New York Times, January 13,...
en.wikipedia.org (parse) The Times of Israel
en.wikipedia.org (imageinfo) File:Times of Israel office, Jerusalem.JPG
en.wikipedia.org (parse) UNIFIL
en.wikipedia.org (imageinfo) File:United Nations Interim Force in...
en.wikipedia.org (parse) United Nations
en.wikipedia.org (parse) United Nations Interim Force in Lebanon
en.wikipedia.org (imageinfo) File:United Nations Interim Force in...
en.wikipedia.org (parse) United Nations peacekeeping
en.wikipedia.org (imageinfo) File:United Nations Peacekeeping Logo.svg
en.wikipedia.org (parse) West Bank areas in the Oslo II Accord
en.wikipedia.org (parse) William R. Higgins
en.wikipedia.org (imageinfo) File:Colonel William-R-Higgins.jpg
en.wikipedia.org (parse) Yitzhak Rabin
en.wikipedia.org (imageinfo) File:Flickr - Israel Defense Forces ...
en.wikipedia.org (parse) Talk:Israeli occupation of Southern Lebanon


skipping Wikipedia:Copying within Wikipedia
skipping Wikipedia:Translation
skipping Template:Cite web
skipping Template:Israeli-occupied territories
skipping Template:Israeli occupations navbox
skipping Template talk:Israeli-occupied territories
skipping Template talk:Israeli occupations navbox
skipping Help:Edit summary
skipping Help:Interlanguage links
skipping Category:CS1 maint: multiple names: authors list
skipping Category:History articles needing translation from Hebrew Wikipedia
skipping Category:Mathematics and art
skipping Category:States and territories disestablished in 2000
skipping Category:States and territories disestablished in the 2000
skipping Category:States and territories established in 1982
skipping Category:States and territories established in the 1982
skipping Category:Use Oxford spelling from April 2022
skipping Category:Use dmy dates from April 2022


In [27]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Beirut,Country,Lebanon
1,Southern Lebanon,-,-
2,Golan Heights,Status,"Syrian territory,two-thirds of which occupied ..."
3,Haifa,Country,"Flagu,Israel"
4,Hasbaya,Country,"Lebanon,Flag"
5,Israel,-,-
6,Jezzine,Country,LBN
7,Jezzine District,Country,"Lebanon,Flag"
8,Jebchit,Country,"Lebanon,Flag"
9,Lebanon,-,-


In [28]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,لبنان,بلد,بيروت
1,-,-,جنوب لبنان
2,"الأراضي السورية (مترجمه),ثلثيها تحتلها إسرائيل...",حالة (مترجمه),هضبة الجولان
3,"علَم (مترجمه),إسرائيل",بلد,حيفا
4,"لبنان ,علم (راية)",بلد,حاصبيا
5,-,-,إسرائيل
6,لبنان (مترجمه),بلد,جزين
7,"لبنان ,علم (راية)",بلد,قضاء جزين
8,"لبنان ,علم (راية)",بلد,جبشيت
9,-,-,لبنان


In [29]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')